In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stop-words/amstopwords.txt
/kaggle/input/gpac-corpus/GPAC.txt


# Loading the data

### Load the stopwords

In [2]:
stop_path = '/kaggle/input/stop-words/amstopwords.txt'
with open(stop_path, 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()
stopwords[-12:], len(stopwords)

(['ፊት', '፣', '፤', 'ቤት', 'በ', 'ም', '፡', 'ዓ', 'የ', 'ሆኖ', 'አሉ', 'አንዱ'], 308)

In [3]:
len(stopwords)

308

### Load the text corpus

In [4]:
import gc

In [5]:
passage_path = '/kaggle/input/gpac-corpus/GPAC.txt'

with open(passage_path, 'r', encoding='utf-8') as file:
    text_corpus = file.read()

print(text_corpus[:100])

    ምን መሰላችሁ? (አንባቢያን) ኢትዮጵያ በተደጋጋሚ ጥሪው ደርሷት ልትታደመው ያልቻለችው የአለም የእግር ኳስ ዋ ለ19ኛ ጊዜ በደቡብ አፍሪካ ሲጠጣ፣ በሩቅ


# Pre-processing the text data

### Remove Stopwords

In [6]:
import re

sentences = re.split(r'[።?]', text_corpus)
sentences = [sentence.strip() for sentence in sentences if sentence.strip()]

del text_corpus
gc.collect()

0

In [7]:
sentences[0]

'ምን መሰላችሁ'

In [8]:
sentences = [sentence.strip() for sentence in sentences if sentence.strip()]

In [9]:
def remove_stopwords_from_sentence(sentence, stopwords):
    words = sentence.split()
    filtered_words = [word for word in words if word not in stopwords]
    return " ".join(filtered_words)

In [10]:
# stop word removal
filtered_sentences = [remove_stopwords_from_sentence(sentence, stopwords) for sentence in sentences]

del sentences
gc.collect()

0

#### Remove characters

In [11]:
def remove_characters(sentence, stopwords):
    s = re.sub(r'[^\w\s]', ' ', sentence)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

In [12]:
sen = "ይህ። ምሳሌ፣ ነው። ፡፡እንዴት፤ነህ?"

c_s = remove_characters(sen, stopwords)

print("Original Sentence:", sen)
print("Cleaned Sentence:", c_s)

Original Sentence: ይህ። ምሳሌ፣ ነው። ፡፡እንዴት፤ነህ?
Cleaned Sentence: ይህ ምሳሌ ነው እንዴት ነህ


In [13]:
# characters removal
filtered_sentences = [remove_characters(sentence, stopwords) for sentence in filtered_sentences]
gc.collect()

0

In [14]:
filtered_sentences[10000]

'ቴዲአፍሮ የግጥም ዝንባሌና ችሎታ እንዳለው አምናለሁ'

In [15]:
# Number of sentences to group together
n = 5

documents = [
    (i, " ".join(filtered_sentences[i:i + n]))
    for i in range(0, len(filtered_sentences), n)
]

In [16]:
import pandas as pd 

In [17]:
document_df = pd.DataFrame(documents, columns=["document_id", "sentences"])

In [18]:
document_df.head()

,document_id,sentences
0,0,መሰላችሁ አንባቢያን በተደጋጋሚ ጥሪው ደርሷት ልትታደመው ያልቻለችው የአለ...
1,5,አይመስለኝም ሃኒ ከውድድሩ ውጭ የሆነችው የቢግ ብራዘርስ አብሮ የመኖር ው...
2,10,ይባላል የፈረንጅ ስምና የፈረንጅ መድሀኒት ያላቸው በሽታዎችም ይይዙን ጀመ...
3,15,ወይስ ኋኧረ ይስጡይ ባለ ንብረት እየነጠቀው ኋባይሆን ለጠንቋይ እሸጠዋለሁ...
4,20,አላቸው እና እሳቸው ኋእሱ እኛን የሆነው ወይስ ነን እሱን የሆንነው ወይስ...


In [19]:
import os

output_dir = "dataset_processed"
os.makedirs(output_dir, exist_ok=True)

document_df.to_csv(os.path.join(output_dir, "documents_processed.csv"), index=False)